### This is the Notebook for Owen Monroe's Homework Assigment 2 for IS-567 Text Mining
##### September 23, 2023

1. Loading the Dataset

In [879]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack



In [880]:
train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')
train_df['review'][24]

nan

In [881]:
# Creates lists and reporting length

tr_rev_list = train_df['review'].to_list()
te_rev_list = test_df['review'].to_list()
tr_rev_list
te_rev_list
print(len(tr_rev_list))
print(len(te_rev_list))


1834
458


In [882]:
# Removing Empties and reporting length
full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

1829
458


2. POS Tagging

In [883]:
# Copying training Data and POS tagging
train_task2_df = full_train_df

# Tokenizing the Reviews
train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/3634727750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)


In [884]:
# Library Download

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [885]:
# POS Tagging and Reporting

train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)
train_task2_df[['review','POS Tagged Tokens']][20:23]

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/121813578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)


,review,POS Tagged Tokens
21,ChatGPT is an impressive AI language model tha...,"[(ChatGPT, NNP), (is, VBZ), (an, DT), (impress..."
22,Though there is a problem that I had found mys...,"[(Though, IN), (there, EX), (is, VBZ), (a, DT)..."
23,"I downloaded the ChatGPT app with high hopes, ...","[(I, PRP), (downloaded, VBD), (the, DT), (Chat..."


3. Extract Unigram Features

In [886]:
# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram to uni-grams
vectorizer = CountVectorizer(ngram_range = (1,1))

# Creating training data representation
train_data_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_cv.shape,"\n") 

# Creating test data representation
test_data_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_cv.shape,"\n") 


(1829, 5551) 

(458, 5551) 



In [887]:
type(train_data_cv)


scipy.sparse._csr.csr_matrix

In [888]:
print(test_data_cv)

  (0, 694)	1
  (0, 2885)	1
  (1, 2233)	1
  (1, 2278)	1
  (1, 3129)	1
  (1, 4898)	1
  (2, 1929)	1
  (2, 2666)	1
  (3, 2645)	1
  (3, 3193)	1
  (3, 5384)	1
  (4, 2170)	1
  (5, 291)	1
  (5, 2135)	1
  (5, 2660)	1
  (5, 3891)	1
  (5, 4932)	1
  (6, 1923)	1
  (7, 291)	1
  (7, 3311)	1
  (8, 2146)	2
  (8, 2660)	1
  (8, 3372)	1
  (8, 4932)	1
  (10, 776)	1
  :	:
  (455, 5437)	1
  (455, 5464)	2
  (455, 5466)	1
  (455, 5503)	5
  (456, 115)	1
  (456, 197)	1
  (456, 230)	1
  (456, 373)	1
  (456, 386)	1
  (456, 388)	1
  (456, 497)	1
  (456, 562)	1
  (456, 701)	1
  (456, 776)	1
  (456, 1435)	1
  (456, 1974)	1
  (456, 2059)	1
  (456, 3303)	1
  (456, 3341)	2
  (456, 4216)	1
  (456, 4507)	1
  (456, 4906)	1
  (456, 4985)	1
  (456, 5403)	1
  (456, 5464)	1


4. Train and Evaluate Classifier

In [889]:
# Define true labels from train set

x_train_cv = train_data_cv
y_train_rev_series = full_train_df["rating"]
x_test_cv = test_data_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv, y_train_rev_series)

# Predict the labels for the test data

predictions = mnb_model.predict(x_test_cv)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 3, 4, 5, 5, 1, 5, 5, 1, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 1, 5, 5, 5, 1, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 1, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,

In [890]:
# Evaluating the Classifier
# Accuracy Classification Score for Unigrams

true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

acc_score = accuracy_score(true_label_list, predict_label_list)
print(f'accuracy score for unigrams = {acc_score}')


accuracy score for unigrams = 0.6331877729257642


In [891]:
# Precision Scores for Unigrams

from sklearn.metrics import precision_score

true_label_array = np.array(true_label_list)
predict_label_array = np.array(predict_label_list)

prec_score_overall = precision_score(true_label_list, predict_label_list, average=None)
prec_score_macro = precision_score(true_label_list, predict_label_list, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list, average='weighted')

print(f'overall precision score for unigrams = {prec_score_overall}')
print(f'macro precision score for unigrams = {prec_score_macro}')
print(f'micro precision score for unigrams = {prec_score_micro}')
print(f'weighted precision scorefor unigrams = {prec_score_weighted}')


overall precision score for unigrams = [0.56565657 0.4        0.13333333 0.38888889 0.69470405]
macro precision score for unigrams = 0.4365165675446049
micro precision score for unigrams = 0.6331877729257642
weighted precision scorefor unigrams = 0.5855492442277733


In [892]:
# Recall Scores for Unigrams 

from sklearn.metrics import recall_score

rec_score_overall = recall_score(true_label_list, predict_label_list, average=None)
rec_score_macro = recall_score(true_label_list, predict_label_list, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list, average='weighted')

print(f'overall macro recall score for unigrams = {rec_score_overall}')
print(f'macro recall score for unigrams = {rec_score_macro}')
print(f'micro recall score for unigrams = {rec_score_micro}')
print(f'weighted recall score for unigrams = {rec_score_weighted}')

overall macro recall score for unigrams = [0.48695652 0.07692308 0.08       0.15909091 0.89919355]
macro recall score for unigrams = 0.34043281122804264
micro recall score for unigrams = 0.6331877729257642
weighted recall score for unigrams = 0.6331877729257642


In [893]:
# F1 Scores for Unigrams

from sklearn.metrics import f1_score

f1_score_overall = f1_score(true_label_list, predict_label_list, average=None)
f1_score_macro = f1_score(true_label_list, predict_label_list, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list, average='weighted')

print(f'overall F1 score for unigrams = {f1_score_overall}')
print(f'macro F1 score for unigrams = {f1_score_macro}')
print(f'micro F1 score for unigrams = {f1_score_micro}')
print(f'weighted F1 score for unigrams = {f1_score_weighted}')


overall F1 score for unigrams = [0.52336449 0.12903226 0.1        0.22580645 0.78383128]
macro F1 score for unigrams = 0.35240689572225525
micro F1 score for unigrams = 0.6331877729257642
weighted F1 score for unigrams = 0.5903218267264796


In [894]:
# Individal Label Reports - Unigrams

from sklearn.metrics import classification_report, confusion_matrix 

print('Individal Label Reports - Unigrams')
print()
print ("Accuracy score: ", accuracy_score(true_label_list, predict_label_list))
print ("Individual label performance: ")
print (classification_report(true_label_list, predict_label_list))
print (confusion_matrix(true_label_list, predict_label_list))

Individal Label Reports - Unigrams

Accuracy score:  0.6331877729257642
Individual label performance: 
              precision    recall  f1-score   support

           1       0.57      0.49      0.52       115
           2       0.40      0.08      0.13        26
           3       0.13      0.08      0.10        25
           4       0.39      0.16      0.23        44
           5       0.69      0.90      0.78       248

    accuracy                           0.63       458
   macro avg       0.44      0.34      0.35       458
weighted avg       0.59      0.63      0.59       458

[[ 56   0   6   4  49]
 [ 13   2   1   1   9]
 [  8   0   2   2  13]
 [  6   0   4   7  27]
 [ 16   3   2   4 223]]


5. Add Bi-grams Features (Continue to Evaluate)

In [895]:
# Extracting Bi-gram features 

# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram range to uni-grams and bigrams
vectorizer = CountVectorizer(ngram_range = (1, 2))

# Creating training data representation
train_data_bigram_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_bigram_cv.shape,"\n") 

# Creating test data representation
test_data_bigram_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_bigram_cv.shape,"\n") 

print(train_data_bigram_cv)

(1829, 41006) 

(458, 41006) 

  (0, 37347)	1
  (0, 35638)	9
  (0, 35036)	5
  (0, 26826)	1
  (0, 38075)	1
  (0, 22562)	1
  (0, 5228)	2
  (0, 37876)	2
  (0, 7119)	2
  (0, 24675)	3
  (0, 22722)	2
  (0, 39767)	1
  (0, 9651)	3
  (0, 14656)	1
  (0, 7449)	1
  (0, 39475)	1
  (0, 18831)	6
  (0, 10297)	1
  (0, 29943)	3
  (0, 28240)	2
  (0, 23095)	1
  (0, 18313)	3
  (0, 27163)	2
  (0, 10007)	1
  (0, 33738)	16
  :	:
  (1826, 16885)	1
  (1827, 18831)	1
  (1827, 4887)	1
  (1827, 39808)	1
  (1827, 17933)	1
  (1827, 25276)	1
  (1827, 13528)	1
  (1827, 4958)	1
  (1827, 20432)	1
  (1827, 7401)	1
  (1827, 1099)	1
  (1827, 27219)	1
  (1827, 16051)	1
  (1827, 18845)	1
  (1827, 1100)	1
  (1827, 39925)	1
  (1827, 20433)	1
  (1827, 7402)	1
  (1827, 13529)	1
  (1827, 17971)	1
  (1827, 27220)	1
  (1827, 25369)	1
  (1828, 18831)	1
  (1828, 14581)	1
  (1828, 19049)	1


In [896]:
# Training a New Naive Bayes Classifier for Unigrams + Bigrams

# Define true labels from train set

x_train_bi_cv = train_data_bigram_cv
y_train_rev_series = full_train_df["rating"]
x_test_bi_cv = test_data_bigram_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_bi_cv, y_train_rev_series)

# Predict the labels for the test data

bi_predictions = mnb_model.predict(x_test_bi_cv)
bi_predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 4, 5, 5, 5, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 4, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

In [897]:
# Simpler Model Evaluation: Unigrams + Bigrams

true_bi_label_list = y_test_rev_series.to_list()
predict_bi_label_list = list(bi_predictions)

print ("Accuracy score for unigrams + bigrams: ", accuracy_score(true_bi_label_list, bi_predictions))

Accuracy score for unigrams + bigrams:  0.6244541484716157


In [898]:
# Precision Scores for Unigrams + Bigram 
bi_prec_score_overall = precision_score(true_bi_label_list, predict_bi_label_list, average=None)
bi_prec_score_macro = precision_score(true_bi_label_list, predict_bi_label_list, average='macro')
bi_prec_score_micro = precision_score(true_bi_label_list, predict_bi_label_list, average='micro')
bi_prec_score_weighted = precision_score(true_bi_label_list, predict_bi_label_list, average='weighted')

print(f'overall precision score with unigrams + bigrams ={bi_prec_score_overall}')
print(f'macro precision score with unigrams + bigrams= {bi_prec_score_macro}')
print(f'micro precision score with unigrams + bigrams = {bi_prec_score_micro}')
print(f'weighted precision score with unigrams + bigrams = {bi_prec_score_weighted}')
# prec_none_array = precision_score(true_bi_label_list, predict_bi_label_list, average=None)



overall precision score with unigrams + bigrams =[0.62820513 0.         0.         0.07692308 0.6519337 ]
macro precision score with unigrams + bigrams= 0.27141238135713275
micro precision score with unigrams + bigrams = 0.6244541484716157
weighted precision score with unigrams + bigrams = 0.5181392208280674


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [899]:
# Recall Scores for Unigrams + Bigrams

bi_rec_score_macro = recall_score(true_bi_label_list, predict_bi_label_list, average='macro')
bi_rec_score_micro = recall_score(true_bi_label_list, predict_bi_label_list, average='micro')
bi_rec_score_weighted = recall_score(true_bi_label_list, predict_bi_label_list, average='weighted')

print(f'macro recall score for unigrams + bigrams = {bi_rec_score_macro}')
print(f'micro recall score for unigrams + bigrams = {bi_rec_score_micro}')
print(f'weighted recall score for unigrams + bigrams = {bi_rec_score_weighted}')
#rec_none_array = recall_score(true_label_array, predict_label_array, average=None)
#rec_none_array


macro recall score for unigrams + bigrams = 0.28008542649496365
micro recall score for unigrams + bigrams = 0.6244541484716157
weighted recall score for unigrams + bigrams = 0.6244541484716157


In [900]:
# F1 Scores for Unigrams + Bigrams

bi_f1_score_macro = f1_score(true_bi_label_list, predict_bi_label_list, average='macro')
bi_f1_score_micro = f1_score(true_bi_label_list, predict_bi_label_list, average='micro')
bi_f1_score_weighted = f1_score(true_bi_label_list, predict_bi_label_list, average='weighted')

print(f'macro F1 score for unigrams + bigrams = {bi_f1_score_macro}')
print(f'micro F1 score for unigrams + bigrams = {bi_f1_score_micro}')
print(f'weighted F1 score for unigrams + bigrams = {bi_f1_score_weighted}')


macro F1 score for unigrams + bigrams = 0.26332604636538254
micro F1 score for unigrams + bigrams = 0.6244541484716157
weighted F1 score for unigrams + bigrams = 0.549853109169772


In [901]:
# Individual Label Reports for Unigrams + Bigrams 

print('Individal Label Reports - Unigrams + Bigrams')
print()
print ("Accuracy score: ", accuracy_score(true_bi_label_list, predict_bi_label_list))
print ("Individual label performance for unigrams + bigrams: ")
print (classification_report(true_bi_label_list, predict_bi_label_list))
print (confusion_matrix(true_bi_label_list, predict_bi_label_list))

Individal Label Reports - Unigrams + Bigrams

Accuracy score:  0.6244541484716157
Individual label performance for unigrams + bigrams: 
              precision    recall  f1-score   support

           1       0.63      0.43      0.51       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.08      0.02      0.04        44
           5       0.65      0.95      0.77       248

    accuracy                           0.62       458
   macro avg       0.27      0.28      0.26       458
weighted avg       0.52      0.62      0.55       458

[[ 49   0   3   3  60]
 [ 11   0   1   3  11]
 [  7   0   0   0  18]
 [  5   0   1   1  37]
 [  6   0   0   6 236]]


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

6. Adding Tri-gram Features

In [902]:
# Extracting Tri-gram features 

# Building Model
train_text = full_train_df["review"]
test_text = full_test_df["review"]

# Setting n-gram range to uni-grams and bigrams
vectorizer = CountVectorizer(ngram_range = (1, 3))

# Creating training data representation
train_data_trigram_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_trigram_cv.shape,"\n") 

# Creating test data representation
test_data_trigram_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_trigram_cv.shape,"\n") 

print(train_data_trigram_cv)

(1829, 97367) 

(458, 97367) 

  (0, 88181)	1
  (0, 83615)	9
  (0, 81998)	5
  (0, 63324)	1
  (0, 90083)	1
  (0, 53192)	1
  (0, 13161)	2
  (0, 89621)	2
  (0, 17504)	2
  (0, 58432)	3
  (0, 53540)	2
  (0, 94178)	1
  (0, 22813)	3
  (0, 33804)	1
  (0, 18396)	1
  (0, 93506)	1
  (0, 44026)	6
  (0, 24127)	1
  (0, 69557)	3
  (0, 66160)	2
  (0, 54558)	1
  (0, 42397)	3
  (0, 63984)	2
  (0, 23554)	1
  (0, 77753)	16
  :	:
  (1827, 18310)	1
  (1827, 2378)	1
  (1827, 64100)	1
  (1827, 37079)	1
  (1827, 44056)	1
  (1827, 2379)	1
  (1827, 94612)	1
  (1827, 48397)	1
  (1827, 18311)	1
  (1827, 31273)	1
  (1827, 41524)	1
  (1827, 64101)	1
  (1827, 60184)	1
  (1827, 44057)	1
  (1827, 2380)	1
  (1827, 94613)	1
  (1827, 48398)	1
  (1827, 18312)	1
  (1827, 12511)	1
  (1827, 31274)	1
  (1827, 41525)	1
  (1827, 64102)	1
  (1828, 44026)	1
  (1828, 33615)	1
  (1828, 44749)	1


In [903]:
# Training another New Naive Bayes Classifier

# Define true labels from train set

x_train_tri_cv = train_data_trigram_cv
y_train_rev_series = full_train_df["rating"]
x_test_tri_cv = test_data_trigram_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_tri_cv, y_train_rev_series)

# Predict the labels for the test data

tri_predictions = mnb_model.predict(x_test_tri_cv)
tri_predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 5, 5, 1, 5, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 4, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 4, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,

In [904]:
# Model Evaluation: Accuracy Score for Unigrams + Bigrams + Trigrams

true_tri_label_list = y_test_rev_series.to_list()
predict_tri_label_list = list(tri_predictions)

print ("Accuracy score for unigrams + bigrams: ", accuracy_score(true_tri_label_list, predict_tri_label_list))

Accuracy score for unigrams + bigrams:  0.6200873362445415


In [905]:
# Precision Scores For Unigrams + Bigrams + Trigrams

tri_prec_score_overall = precision_score(true_tri_label_list, predict_tri_label_list, average=None)
tri_prec_score_macro = precision_score(true_tri_label_list, predict_tri_label_list, average='macro')
tri_prec_score_micro = precision_score(true_tri_label_list, predict_tri_label_list, average='micro')
tri_prec_score_weighted = precision_score(true_tri_label_list, predict_tri_label_list, average='weighted')

print(f'overall precision score for unigrams + bigrams + trigrams = {tri_prec_score_overall}')
print(f'macro precision score for unigrams + bigrams + trigrams = {tri_prec_score_macro}')
print(f'micro precision score for unigrams + bigrams + trigrams = {tri_prec_score_micro}')
print(f'weighted precision score for unigrams + bigrams + trigrams = {tri_prec_score_weighted}')



overall precision score for unigrams + bigrams + trigrams = [0.64705882 0.         0.         0.         0.63829787]
macro precision score for unigrams + bigrams + trigrams = 0.2570713391739675
micro precision score for unigrams + bigrams + trigrams = 0.6200873362445415
weighted precision score for unigrams + bigrams + trigrams = 0.5080996442059124


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [906]:
# Recall Scores for Unigrams + Bigrams + Trigrams

tri_rec_score_overall = recall_score(true_tri_label_list, predict_tri_label_list, average=None)
tri_rec_score_macro = recall_score(true_tri_label_list, predict_tri_label_list, average='macro')
tri_rec_score_micro = recall_score(true_tri_label_list, predict_tri_label_list, average='micro')
tri_rec_score_weighted = recall_score(true_tri_label_list, predict_tri_label_list, average='weighted')

print(f'overall recall score for unigrams + bigrams + trigrams = {tri_rec_score_overall}')
print(f'macro recall score for unigrams + bigrams + trigrams = {tri_rec_score_macro}')
print(f'micro recall score for unigrams + bigrams + trigrams = {tri_rec_score_micro}')
print(f'weighted recall score for unigrams + bigrams + trigrams = {tri_rec_score_weighted}')
#rec_none_array = recall_score(true_label_array, predict_label_array, average=None)
#rec_none_array


overall recall score for unigrams + bigrams + trigrams = [0.3826087  0.         0.         0.         0.96774194]
macro recall score for unigrams + bigrams + trigrams = 0.270070126227209
micro recall score for unigrams + bigrams + trigrams = 0.6200873362445415
weighted recall score for unigrams + bigrams + trigrams = 0.6200873362445415


In [907]:
# F1 Scores for Unigrams + Bigrams + Trigrams

tri_f1_score_overall = f1_score(true_tri_label_list, predict_tri_label_list, average=None)
tri_f1_score_macro = f1_score(true_tri_label_list, predict_tri_label_list, average='macro')
tri_f1_score_micro = f1_score(true_tri_label_list, predict_tri_label_list, average='micro')
tri_f1_score_weighted = f1_score(true_tri_label_list, predict_tri_label_list, average='weighted')

print(f'overall F1 score for unigrams + bigrams + trigrams = {tri_f1_score_overall}')
print(f'macro F1 score for unigrams + bigrams + trigrams = {tri_f1_score_macro}')
print(f'micro F1 score for unigrams + bigrams + trigrams = {tri_f1_score_micro}')
print(f'weighted F1 score for unigrams + bigrams + trigrams = {tri_f1_score_weighted}')

overall F1 score for unigrams + bigrams + trigrams = [0.48087432 0.         0.         0.         0.76923077]
macro F1 score for unigrams + bigrams + trigrams = 0.250021017234132
micro F1 score for unigrams + bigrams + trigrams = 0.6200873362445415
weighted F1 score for unigrams + bigrams + trigrams = 0.5372702559330093


In [908]:
# Individual Label Reports for Unigrams + Bigrams + Trigrams

print('Individal Label Reports - Unigrams + Bigrams + Trigrams')
print()
print ("Accuracy score: ", accuracy_score(true_tri_label_list, predict_tri_label_list))
print ("Individual label performance for unigrams + bigrams + trigrams: ")
print (classification_report(true_tri_label_list, predict_tri_label_list))
print (confusion_matrix(true_tri_label_list, predict_tri_label_list))

Individal Label Reports - Unigrams + Bigrams + Trigrams

Accuracy score:  0.6200873362445415
Individual label performance for unigrams + bigrams + trigrams: 
              precision    recall  f1-score   support

           1       0.65      0.38      0.48       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        44
           5       0.64      0.97      0.77       248

    accuracy                           0.62       458
   macro avg       0.26      0.27      0.25       458
weighted avg       0.51      0.62      0.54       458

[[ 44   0   3   3  65]
 [ 10   0   1   2  13]
 [  6   0   0   0  19]
 [  4   0   1   0  39]
 [  4   0   0   4 240]]


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [909]:
''' # Evaluating Labels: 1,  Unigrams + Bigrams + Trigrams

# Constructing Filter for Label == 1
tri_label_1 = 1 

tri_label_filter = [1 if label == tri_label_1 else 0 for label in true_tri_label_list]

true_tri_labels_for_label_1 = [true_tri_label_list[i] for i, mask in enumerate(tri_label_filter) if mask == 1]
predicted_tri_labels_for_lable_1 = [predict_tri_label_list[i] for i, mask in enumerate(tri_label_filter) if mask == 1]

# Accuracy Scoring for New List
accuracy_for_tri_label_1 = accuracy_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1)

# Precision Scoring for New List
prec_score_macro_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='macro')
prec_score_micro_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='micro')
prec_score_weighted_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='weighted')

# Recall Scoring for New List
rec_score_macro_tri_label_1 = recall_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='macro')
rec_score_micro_tri_label_1 = recall_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='micro')
rec_score_weighted_tri_label_1 = recall_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='weighted')

#F1 Scoring for New List
f1_score_macro_tri_label_1 = f1_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='macro')
f1_score_micro_tri_label_1 = f1_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='micro')
f1_score_weighted_tri_label_1 = f1_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='weighted')


# Reporting All 
print('Reporting Scores for Label 1 - unigrams + bigrams + trigrams')
print()
print(f'Accuracy for label {tri_label_1}: {accuracy_for_tri_label_1:.2f}')
print()
print(f'macro precision score for label {tri_label_1} = {prec_score_macro_tri_label_1}')
print(f'micro precision score for label {tri_label_1} = {prec_score_micro_tri_label_1}')
print(f'weighted precision score for label {tri_label_1} = {prec_score_weighted_tri_label_1}')
print()
print(f'macro recall score for label {tri_label_1} = {rec_score_macro_tri_label_1}')
print(f'micro recall score for label {tri_label_1} = {rec_score_micro_tri_label_1}')
print(f'weighted recall score for label {tri_label_1} = {rec_score_weighted_tri_label_1}')
print()
print(f'macro F1 score for label {tri_label_1} = {f1_score_macro_tri_label_1}')
print(f'micro F1 score for label {tri_label_1} = {f1_score_micro_tri_label_1}')
print(f'weighted F1 score for label {tri_label_1} = {f1_score_weighted_tri_label_1}') '''


" # Evaluating Labels: 1,  Unigrams + Bigrams + Trigrams\n\n# Constructing Filter for Label == 1\ntri_label_1 = 1 \n\ntri_label_filter = [1 if label == tri_label_1 else 0 for label in true_tri_label_list]\n\ntrue_tri_labels_for_label_1 = [true_tri_label_list[i] for i, mask in enumerate(tri_label_filter) if mask == 1]\npredicted_tri_labels_for_lable_1 = [predict_tri_label_list[i] for i, mask in enumerate(tri_label_filter) if mask == 1]\n\n# Accuracy Scoring for New List\naccuracy_for_tri_label_1 = accuracy_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1)\n\n# Precision Scoring for New List\nprec_score_macro_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='macro')\nprec_score_micro_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='micro')\nprec_score_weighted_tri_label_1 = precision_score(true_tri_labels_for_label_1, predicted_tri_labels_for_lable_1, average='weigh

7. Add TF IDF Features

In [910]:
# Unigrams seems to have highest F1 / macro F1 score

from sklearn.feature_extraction.text import TfidfVectorizer

# this time, we vectorize using TF-IDF
text = full_train_df["review"]
print(text,"\n")

tf = TfidfVectorizer()
tf.fit(text)

print(tf.vocabulary_,"\n")

# encode document
data = tf.transform(text)

# summarize encoded vector
print(data.shape,"\n") 
print(data.toarray())

0       Up to this point I?€?ve mostly been using Chat...
1       I?€?ve been a user since it?€?s initial roll o...
2       This app would almost be perfect if it wasn?€?...
3       I recently downloaded the app and overall, it'...
4       I appreciate the devs implementing Siri suppor...
                              ...                        
1829                   Please make a iPad version of this
1830                                             Best app
1831                 Why it?€?s not available in Ethiopia
1832    It agrees with letting children be forced into...
1833                                          It?€?s good
Name: review, Length: 1829, dtype: object 

{'up': 5224, 'to': 4985, 'this': 4932, 'point': 3613, 've': 5282, 'mostly': 3134, 'been': 493, 'using': 5253, 'chatgpt': 776, 'on': 3331, 'my': 3165, 'windows': 5454, 'desktop': 1302, 'google': 2139, 'chrome': 806, 'while': 5421, 'it': 2666, 'doable': 1426, 'screen': 4267, 'reader': 3883, 'navigation': 3184, 'is': 26

In [911]:
train_data_tfidf = tf.fit_transform(train_text)
print(train_data_tfidf.shape,"\n") 

test_data_tfidf = tf.transform(test_text)
print(test_data_tfidf.shape,"\n") 

idf = tf.idf_

# print out feature names (the words) and the IDF values
print(dict(zip(tf.get_feature_names(), idf)))

(1829, 5551) 

(458, 5551) 

{'10': 5.567632266669026, '100': 6.902633333401366, '101': 7.818924065275521, '11': 7.125776884715576, '12': 5.804021044733257, '128': 7.818924065275521, '12pro': 7.413458957167357, '13': 6.0271645960474665, '13pro': 7.818924065275521, '14': 5.216234379831137, '15': 6.720311776607412, '156': 7.818924065275521, '16': 5.94712188837393, '17': 7.818924065275521, '18': 7.413458957167357, '19': 7.125776884715576, '1940': 7.818924065275521, '1949': 7.818924065275521, '20': 5.376577029906317, '2007': 7.818924065275521, '2020': 7.818924065275521, '2021': 4.798499179131159, '2022': 6.902633333401366, '2023': 6.432629704155631, '20mins': 7.413458957167357, '21': 7.818924065275521, '23': 7.818924065275521, '24': 7.125776884715576, '25': 6.902633333401366, '27': 7.818924065275521, '2nd': 7.413458957167357, '30': 6.720311776607412, '31': 7.818924065275521, '32k': 7.818924065275521, '35': 7.818924065275521, '360': 7.818924065275521, '3d': 7.818924065275521, '3rd': 6.90263

In [912]:
# define true labels from train set
x_train_tfidf = train_data_tfidf
y_train_tfidf_series = full_train_df["rating"]
x_test_tfidf = test_data_tfidf
y_test_tfidf_series = full_test_df["rating"]


model = MultinomialNB()
model.fit(x_train_tfidf, y_train_tfidf_series)
tfidf_predictions = model.predict(x_test_tfidf)

from sklearn.metrics import classification_report, confusion_matrix 

print ("Accuracy score: ", accuracy_score(y_test_tfidf_series, tfidf_predictions))
print ("Individual label performance: ")
print (classification_report(y_test_tfidf_series, tfidf_predictions))
print (confusion_matrix(y_test_tfidf_series, tfidf_predictions))

Accuracy score:  0.5829694323144105
Individual label performance: 
              precision    recall  f1-score   support

           1       0.67      0.17      0.28       115
           2       0.00      0.00      0.00        26
           3       0.00      0.00      0.00        25
           4       0.00      0.00      0.00        44
           5       0.58      1.00      0.73       248

    accuracy                           0.58       458
   macro avg       0.25      0.23      0.20       458
weighted avg       0.48      0.58      0.46       458

[[ 20   0   0   0  95]
 [  3   0   0   0  23]
 [  3   0   0   0  22]
 [  3   0   0   0  41]
 [  1   0   0   0 247]]


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

8. Building Model with other Columns

In [913]:
# Unigrams still appears to be the best system


train_title_text = full_train_df["title"]
print(train_title_text,"\n")

tf = TfidfVectorizer()
tf.fit(train_title_text)

print(tf.vocabulary_,"\n")

# encode document
title_data = tf.transform(train_title_text)

# summarize encoded vector
print(title_data.shape,"\n") 
print(title_data.toarray())


0       Much more accessible for blind users than the ...
1                    Much anticipated, wasn?€?t let down.
2               Almost 5 stars, but?€? no search function
3                                 4.5 stars, here?€?s why
4       Good, but Siri support would take it to the ne...
                              ...                        
1829                                            pad os???
1830                                           Goated app
1831                                                   Co
1832                                    Crazy world views
1833                                             Good app
Name: title, Length: 1829, dtype: object 

{'much': 840, 'more': 835, 'accessible': 28, 'for': 486, 'blind': 149, 'users': 1326, 'than': 1238, 'the': 1242, 'web': 1368, 'version': 1340, 'anticipated': 74, 'wasn': 1362, 'let': 737, 'down': 361, 'almost': 59, 'stars': 1177, 'but': 182, 'no': 863, 'search': 1104, 'function': 507, 'here': 592, 'why': 1387, 'good':

In [914]:
test_title_text = full_test_df['title']

train_title_tfidf = tf.fit_transform(train_title_text)
print(train_title_tfidf.shape,"\n") 

test_title_tfidf = tf.transform(test_title_text)
print(test_title_tfidf.shape,"\n") 

title_idf = tf.idf_

# print out feature names (the words) and the IDF values
print(dict(zip(tf.get_feature_names(), title_idf)))

(1829, 1435) 

(458, 1435) 

{'10': 6.432629704155631, '100': 7.818924065275521, '12': 7.413458957167357, '14': 6.902633333401366, '14pm': 7.818924065275521, '14pro': 7.818924065275521, '15': 7.818924065275521, '16': 7.413458957167357, '20': 7.413458957167357, '2021': 7.818924065275521, '2023': 7.413458957167357, '21': 7.818924065275521, '23': 7.818924065275521, '2nd': 7.818924065275521, '2sv': 7.818924065275521, '30': 7.413458957167357, '35': 7.818924065275521, '3d': 7.818924065275521, '403': 7.818924065275521, '4th': 7.818924065275521, '720th': 7.818924065275521, '75': 7.818924065275521, '99': 7.818924065275521, 'able': 7.413458957167357, 'about': 7.125776884715576, 'absolutely': 6.902633333401366, 'accept': 7.413458957167357, 'access': 5.67885790177925, 'accessible': 6.720311776607412, 'accessing': 7.818924065275521, 'account': 5.144775415848993, 'accounts': 7.818924065275521, 'accuracy': 7.818924065275521, 'accurate': 7.413458957167357, 'activate': 7.818924065275521, 'actual': 7.41

In [915]:
# define true labels from train set
x_train_title_tfidf = train_title_tfidf
y_train_title_tfidf_series = full_train_df["rating"]
x_test_title_tfidf = test_title_tfidf
y_test_title_tfidf_series = full_test_df["rating"]


model = MultinomialNB()
model.fit(x_train_title_tfidf, y_train_title_tfidf_series)
tfidf_title_predictions = model.predict(x_test_title_tfidf)

from sklearn.metrics import classification_report, confusion_matrix 

print ("Accuracy score: ", accuracy_score(y_test_title_tfidf_series, tfidf_title_predictions))
print ("Individual label performance: ")
print (classification_report(y_test_title_tfidf_series, tfidf_title_predictions))
print (confusion_matrix(y_test_title_tfidf_series, tfidf_title_predictions))

Accuracy score:  0.6069868995633187
Individual label performance: 
              precision    recall  f1-score   support

           1       0.65      0.29      0.40       115
           2       0.00      0.00      0.00        26
           3       0.33      0.04      0.07        25
           4       0.25      0.02      0.04        44
           5       0.61      0.98      0.75       248

    accuracy                           0.61       458
   macro avg       0.37      0.27      0.25       458
weighted avg       0.53      0.61      0.51       458

[[ 33   0   1   2  79]
 [  5   0   1   0  20]
 [  5   0   1   1  18]
 [  3   0   0   1  40]
 [  5   0   0   0 243]]


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [982]:
# Title + Review Training Dataframe 

full_train_df['title and review'] = full_train_df['title'] + ' ' + full_train_df['review']

full_train_df


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/3483630528.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['title and review'] = full_train_df['title'] + ' ' + full_train_df['review']


,date,title,review,rating,NLTK Tokenized Reviews,POS Tagged Tokens,three rating,title and review
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,"[Up, to, this, point, I, ?, €, ?, ve, mostly, ...","[(Up, RB), (to, TO), (this, DT), (point, NN), ...",positive,Much more accessible for blind users than the ...
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,"[I, ?, €, ?, ve, been, a, user, since, it, ?, ...","[(I, PRP), (?, .), (€, VB), (?, .), (ve, FW), ...",positive,"Much anticipated, wasn?€?t let down. I?€?ve be..."
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,"[This, app, would, almost, be, perfect, if, it...","[(This, DT), (app, NN), (would, MD), (almost, ...",positive,"Almost 5 stars, but?€? no search function This..."
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,"[I, recently, downloaded, the, app, and, overa...","[(I, PRP), (recently, RB), (downloaded, VBD), ...",positive,"4.5 stars, here?€?s why I recently downloaded ..."
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,"[I, appreciate, the, devs, implementing, Siri,...","[(I, PRP), (appreciate, VBP), (the, DT), (devs...",positive,"Good, but Siri support would take it to the ne..."
...,...,...,...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,"[Please, make, a, iPad, version, of, this]","[(Please, NNP), (make, VB), (a, DT), (iPad, JJ...",negative,pad os??? Please make a iPad version of this
1830,5/19/2023 1:23,Goated app,Best app,5,"[Best, app]","[(Best, JJS), (app, NN)]",positive,Goated app Best app
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,"[Why, it, ?, €, ?, s, not, available, in, Ethi...","[(Why, WRB), (it, PRP), (?, .), (€, VB), (?, ....",negative,Co Why it?€?s not available in Ethiopia
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,"[It, agrees, with, letting, children, be, forc...","[(It, PRP), (agrees, VBZ), (with, IN), (lettin...",negative,Crazy world views It agrees with letting child...


In [983]:
# Title + Review Test Dataframe 

full_test_df['title and review'] = full_test_df['title'] + ' ' + full_test_df['review']

full_test_df

,date,title,review,rating,three rating,title and review
0,5/19/2023 6:09,error unsupported country,cant login,2,negative,error unsupported country cant login
1,5/19/2023 9:39,Hype junk,More harm than help.,1,negative,Hype junk More harm than help.
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1,negative,your gpt 4 is fake Fix it
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5,positive,Please impose IPadOS We need IPadOS!!!
4,5/19/2023 20:49,Amazing,Great,5,positive,Amazing Great
...,...,...,...,...,...,...
453,5/19/2023 0:17,Andrew Justino Wilson 5/19/23,This has to be a beginning to something crazy ...,5,positive,Andrew Justino Wilson 5/19/23 This has to be a...
454,5/18/2023 19:13,Superb AI,I?€?ve been using chat and have been a proud p...,5,positive,Superb AI I?€?ve been using chat and have been...
455,5/18/2023 18:27,Fantastic App with Room for Enhancements,The ChatGPT iOS app is an outstanding product....,5,positive,Fantastic App with Room for Enhancements The C...
456,5/18/2023 17:17,"Awesome technology, deplorable tactics",Sam Altman?€?s blatant attempt at regulatory c...,2,negative,"Awesome technology, deplorable tactics Sam Alt..."


In [997]:
# Unigram Extraction and NB Model is the best system with the highest F1 score it seems

# Building Model

train_title_rev_text = full_train_df["title and review"]
test_title_rev_text = full_test_df["title and review"]

# Setting n-gram range to uni-grams
vectorizer = CountVectorizer(ngram_range = (1, 1))

# Creating training data representation
train_data_title_rev_cv = vectorizer.fit_transform(train_title_rev_text.values.astype('U'))
print(train_data_bigram_cv.shape,"\n") 

# Creating test data representation
test_data_title_rev_cv = vectorizer.transform(test_title_rev_text.values.astype('U'))
print(test_data_title_rev_cv.shape,"\n") 

print(train_data_title_rev_cv)


(1829, 41006) 

(458, 5742) 

  (0, 3261)	2
  (0, 3242)	1
  (0, 88)	2
  (0, 2038)	1
  (0, 584)	1
  (0, 5425)	2
  (0, 5062)	1
  (0, 5070)	17
  (0, 5572)	1
  (0, 5478)	1
  (0, 5395)	1
  (0, 5149)	9
  (0, 5096)	5
  (0, 3740)	1
  (0, 5459)	1
  (0, 3248)	1
  (0, 510)	2
  (0, 5427)	2
  (0, 802)	2
  (0, 3450)	3
  (0, 3279)	2
  (0, 5638)	1
  (0, 1346)	3
  (0, 2215)	1
  (0, 833)	1
  :	:
  (1825, 2204)	1
  (1826, 2762)	1
  (1826, 2545)	1
  (1826, 5617)	1
  (1826, 426)	1
  (1826, 3365)	1
  (1826, 875)	1
  (1826, 1748)	1
  (1827, 2762)	1
  (1827, 491)	1
  (1827, 5650)	1
  (1827, 2711)	1
  (1827, 3492)	1
  (1827, 5681)	1
  (1827, 1139)	1
  (1827, 2040)	1
  (1827, 5494)	1
  (1827, 2908)	1
  (1827, 821)	1
  (1827, 201)	1
  (1827, 3828)	1
  (1827, 2414)	1
  (1828, 2762)	1
  (1828, 2211)	2
  (1828, 300)	1


In [998]:
# Training another New Naive Bayes Classifier

# Define true labels from train set

x_train_title_rev_cv = train_data_title_rev_cv
y_train_title_rev_series = full_train_df["rating"]
x_test_title_rev_cv = test_data_title_rev_cv
y_test_title_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_title_rev_cv, y_train_title_rev_series)

# Predict the labels for the test data

title_rev_predictions = mnb_model.predict(x_test_title_rev_cv)
title_rev_predictions

array([1, 1, 5, 3, 5, 5, 5, 5, 5, 1, 5, 3, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 1, 3, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 1, 5, 5,
       1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 5, 1, 5, 5, 3, 1, 5,
       5, 5, 5, 5, 5, 3, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 3, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 4, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 4, 5, 2, 5, 2, 1, 2, 5, 5, 5, 1, 1, 5, 5, 5, 5,
       5, 5, 4, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 4, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,

In [999]:
# Evaluating Title + Review Classifier

print ("Accuracy score: ", accuracy_score(y_test_title_rev_series, title_rev_predictions))
print ("Individual label performance: ")
print (classification_report(y_test_title_rev_series, title_rev_predictions))
print (confusion_matrix(y_test_title_rev_series, title_rev_predictions))

Accuracy score:  0.6419213973799127
Individual label performance: 
              precision    recall  f1-score   support

           1       0.58      0.50      0.53       115
           2       0.29      0.08      0.12        26
           3       0.09      0.08      0.09        25
           4       0.48      0.23      0.31        44
           5       0.72      0.90      0.80       248

    accuracy                           0.64       458
   macro avg       0.43      0.36      0.37       458
weighted avg       0.60      0.64      0.61       458

[[ 57   2   6   4  46]
 [ 10   2   5   2   7]
 [ 11   0   2   2  10]
 [  7   0   4  10  23]
 [ 14   3   5   3 223]]


10. Error Analysis

In [984]:
predictions = predictions
labels = full_test_df["rating"]
inputs = full_test_df["review"]


for idx, prediction, label in zip(enumerate(inputs), predictions, labels):
    if prediction != label:
        print("Sample", 'has been classified as', prediction, 'and should be', label, idx) 
        print ("\n")

Sample has been classified as 1 and should be 2 (0, 'cant login')


Sample has been classified as 5 and should be 1 (1, 'More harm than help.')


Sample has been classified as 5 and should be 4 (6, 'First')


Sample has been classified as 5 and should be 3 (9, '????')


Sample has been classified as 5 and should be 1 (10, 'Old version of ChatGPT.')


Sample has been classified as 3 and should be 1 (11, 'Unusable on iPad Pro currently.')


Sample has been classified as 1 and should be 5 (12, 'what the title says ^')


Sample has been classified as 1 and should be 2 (13, 'i subscribe the plus for gpt4???but just get gpt3.0 with fake label 4.0.release real gpt4 now?????????')


Sample has been classified as 5 and should be 1 (23, '..')


Sample has been classified as 5 and should be 2 (24, 'This app is based on chatgpt 3 (cutoff date Sep 2021)')


Sample has been classified as 4 and should be 1 (30, 'need iPad version????????????????')


Sample has been classified as 5 and should be 1 (31

11. Three-Way vs Five-Way Classification

In [985]:
def change_five_to_three(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        return 'unknown'


In [986]:
#ratings_train_df = pd.DataFrame()

three_rate_list = full_train_df['rating'].apply(change_five_to_three).to_list()

three_rate_list
#ratings_train_df['three rating'] = three_rate_list

# ratings_train_df['five ratings'] = full_train_df['rating']

#ratings_train_df


['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'neutral',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'neutral',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'n

In [987]:
three_rate_test_list = full_test_df['rating'].apply(change_five_to_three).to_list()

three_rate_test_list

['negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 '

In [988]:

full_test_df['three rating'] = three_rate_test_list

full_test_df

,date,title,review,rating,three rating,title and review
0,5/19/2023 6:09,error unsupported country,cant login,2,negative,error unsupported country cant login
1,5/19/2023 9:39,Hype junk,More harm than help.,1,negative,Hype junk More harm than help.
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1,negative,your gpt 4 is fake Fix it
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5,positive,Please impose IPadOS We need IPadOS!!!
4,5/19/2023 20:49,Amazing,Great,5,positive,Amazing Great
...,...,...,...,...,...,...
453,5/19/2023 0:17,Andrew Justino Wilson 5/19/23,This has to be a beginning to something crazy ...,5,positive,Andrew Justino Wilson 5/19/23 This has to be a...
454,5/18/2023 19:13,Superb AI,I?€?ve been using chat and have been a proud p...,5,positive,Superb AI I?€?ve been using chat and have been...
455,5/18/2023 18:27,Fantastic App with Room for Enhancements,The ChatGPT iOS app is an outstanding product....,5,positive,Fantastic App with Room for Enhancements The C...
456,5/18/2023 17:17,"Awesome technology, deplorable tactics",Sam Altman?€?s blatant attempt at regulatory c...,2,negative,"Awesome technology, deplorable tactics Sam Alt..."


In [989]:
full_train_df['three rating'] = three_rate_list

full_train_df

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/2342193202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['three rating'] = three_rate_list


,date,title,review,rating,NLTK Tokenized Reviews,POS Tagged Tokens,three rating,title and review
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,"[Up, to, this, point, I, ?, €, ?, ve, mostly, ...","[(Up, RB), (to, TO), (this, DT), (point, NN), ...",positive,Much more accessible for blind users than the ...
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,"[I, ?, €, ?, ve, been, a, user, since, it, ?, ...","[(I, PRP), (?, .), (€, VB), (?, .), (ve, FW), ...",positive,"Much anticipated, wasn?€?t let down. I?€?ve be..."
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,"[This, app, would, almost, be, perfect, if, it...","[(This, DT), (app, NN), (would, MD), (almost, ...",positive,"Almost 5 stars, but?€? no search function This..."
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,"[I, recently, downloaded, the, app, and, overa...","[(I, PRP), (recently, RB), (downloaded, VBD), ...",positive,"4.5 stars, here?€?s why I recently downloaded ..."
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,"[I, appreciate, the, devs, implementing, Siri,...","[(I, PRP), (appreciate, VBP), (the, DT), (devs...",positive,"Good, but Siri support would take it to the ne..."
...,...,...,...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,"[Please, make, a, iPad, version, of, this]","[(Please, NNP), (make, VB), (a, DT), (iPad, JJ...",negative,pad os??? Please make a iPad version of this
1830,5/19/2023 1:23,Goated app,Best app,5,"[Best, app]","[(Best, JJS), (app, NN)]",positive,Goated app Best app
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,"[Why, it, ?, €, ?, s, not, available, in, Ethi...","[(Why, WRB), (it, PRP), (?, .), (€, VB), (?, ....",negative,Co Why it?€?s not available in Ethiopia
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,"[It, agrees, with, letting, children, be, forc...","[(It, PRP), (agrees, VBZ), (with, IN), (lettin...",negative,Crazy world views It agrees with letting child...


In [990]:
# Naive Bayes Classifier for the Three Way Rating

# Define true labels from train set

x_train_three_cv = train_data_cv
y_train_three_series = full_train_df['three rating']
x_test_three_cv = test_data_cv
y_test_three_series = full_test_df['three rating']

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_three_cv, y_train_three_series)

# Predict the labels for the test data

three_predictions = mnb_model.predict(x_test_three_cv)
three_predictions

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive'

In [991]:
true_label_three_list = y_test_three_series.to_list()
predict_label_three_list = list(three_predictions)

acc_three_score = accuracy_score(true_label_three_list, predict_label_three_list)
print(f'accuracy score for three-way classification = {acc_three_score}')


accuracy score for three-way classification = 0.7358078602620087


In [992]:
# Precision Scores for Three-Way Classification

from sklearn.metrics import precision_score


three_prec_score_overall = precision_score(true_label_three_list, predict_label_three_list, average=None)
three_prec_score_macro = precision_score(true_label_three_list, predict_label_three_list, average='macro')
three_prec_score_micro = precision_score(true_label_three_list, predict_label_three_list, average='micro')
three_prec_score_weighted = precision_score(true_label_three_list, predict_label_three_list, average='weighted')

print(f'overall precision score for three-way classification = {three_prec_score_overall}')
print(f'macro precision score for three-way classification = {three_prec_score_macro}')
print(f'micro precision score for three-way classification = {three_prec_score_micro}')
print(f'weighted precision scorefor three-way classification = {three_prec_score_weighted}')


overall precision score for three-way classification = [0.68807339 0.16666667 0.77151335]
macro precision score for three-way classification = 0.5420844714259355
micro precision score for three-way classification = 0.7358078602620087
weighted precision scorefor three-way classification = 0.7128098567692405


In [993]:
three_rec_score_overall = recall_score(true_label_three_list, predict_label_three_list, average=None)
three_rec_score_macro = recall_score(true_label_three_list, predict_label_three_list, average='macro')
three_rec_score_micro = recall_score(true_label_three_list, predict_label_three_list, average='micro')
three_rec_score_weighted = recall_score(true_label_three_list, predict_label_three_list, average='weighted')

print(f'overall macro recall score for three-way classification = {three_rec_score_overall}')
print(f'macro recall score for three-way classification = {three_rec_score_macro}')
print(f'micro recall score for three-way classification = {three_rec_score_micro}')
print(f'weighted recall score for three-way classification = {three_rec_score_weighted}')

overall macro recall score for three-way classification = [0.53191489 0.08       0.89041096]
macro recall score for three-way classification = 0.5007752841737103
micro recall score for three-way classification = 0.7358078602620087
weighted recall score for three-way classification = 0.7358078602620087


In [994]:
three_f1_score_overall = f1_score(true_label_three_list, predict_label_three_list, average=None)
three_f1_score_macro = f1_score(true_label_three_list, predict_label_three_list, average='macro')
three_f1_score_micro = f1_score(true_label_three_list, predict_label_three_list, average='micro')
three_f1_score_weighted = f1_score(true_label_three_list, predict_label_three_list, average='weighted')

print(f'overall F1 score for three-way classification = {three_f1_score_overall}')
print(f'macro F1 score for three-way classification = {three_f1_score_macro}')
print(f'micro F1 score for three-way classification = {three_f1_score_micro}')
print(f'weighted F1 score for three-way classification = {three_f1_score_weighted}')

overall F1 score for three-way classification = [0.6        0.10810811 0.82670906]
macro F1 score for three-way classification = 0.5116057233704292
micro F1 score for three-way classification = 0.7358078602620087
weighted F1 score for three-way classification = 0.7176894078769239


In [995]:
print('Individal Label Reports - Three-Way Classification')
print()
print ("Accuracy score: ", accuracy_score(true_label_three_list, predict_label_three_list))
print ("Individual label performance: ")
print (classification_report(true_label_three_list, predict_label_three_list))
print (confusion_matrix(true_label_three_list, predict_label_three_list))

Individal Label Reports - Three-Way Classification

Accuracy score:  0.7358078602620087
Individual label performance: 
              precision    recall  f1-score   support

    negative       0.69      0.53      0.60       141
     neutral       0.17      0.08      0.11        25
    positive       0.77      0.89      0.83       292

    accuracy                           0.74       458
   macro avg       0.54      0.50      0.51       458
weighted avg       0.71      0.74      0.72       458

[[ 75   4  62]
 [  8   2  15]
 [ 26   6 260]]
